# Team Choice Algorithm
In this notebook, we are going to build an algorithm that selects the optimal team (i.e. highest points with the 100M budget) based on already predicted points from our internal and/or external Machine Learning models. This will be a zero-one knapsack algorithm that has to fulfil the following constraints:
- Choose **15 players** where ***2 are goalkeepers***, ***5 are defenders***, ***5 are midfielders***, and ***3 are strikers***
- Your 15-player choice should **NOT include** more than 3 players from the same team
- Maximise the points for **11 players eligible to play** per gameweek where ***1 is a goalkeeper***, ***3-5 are defenders***, ***3-5 are midfielders***, and ***1-3 are strikers***
- Note players on the bench will be included if one eligible player is injured/does not play in a gameweek
- You can make one transfer every gameweek and this can be carried forward but never exceeds 2 free transfers. All additional transfers cost 4 points on your gameweek score

In [1]:
import pandas as pd

In [18]:
predictions = pd.read_csv("predictedpoints.csv")
predictions.head()

,ID,Name,Team,Pos,Price,14_pts,15_pts,16_pts,17_pts,18_pts,19_pts,20_pts
0,1,Fábio Vieira,ARS,MID,5.4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2,G.Jesus,ARS,FWD,6.8,1.81006,1.69101,1.99226,1.71490,1.87430,1.66682,1.62687
2,3,Gabriel,ARS,DEF,6.2,1.80719,2.60986,2.58463,2.90901,2.66428,3.11601,2.87315
3,4,Havertz,ARS,FWD,7.9,2.46165,2.24302,2.59427,2.24465,2.51543,2.22071,2.17817
4,5,Hein,ARS,GK,4.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [25]:
start_gw = 14 #Enter the starting gameweek for your team selection
gw_count = 5 #Up to how many gameweeks in the future would you like to consider

## Compute the total expected points for the next gw_count weeks  the starting gameweek
predictions['total_pts'] = predictions[str(start_gw) + '_pts']
for gw in range(start_gw+1,start_gw+gw_count):
    predictions['total_pts'] += predictions[str(gw) + '_pts']

predictions = predictions.sort_values(by='total_pts',ascending=False) #Sort by total predicted points
predictions = predictions[predictions.total_pts != 0] #Drop all players with 0 expected points
predictions.head(10)

,ID,Name,Team,Pos,Price,14_pts,15_pts,16_pts,17_pts,18_pts,19_pts,20_pts,total_pts
16,17,Saka,ARS,MID,10.3,4.80651,4.52891,5.52161,4.49806,5.98499,4.95165,4.56745,25.34008
327,328,M.Salah,LIV,MID,13.1,4.27397,4.45441,5.10245,4.80723,6.26130,5.40262,4.93968,24.89936
350,351,Haaland,MCI,FWD,15.0,5.39160,4.30897,5.01468,4.02910,5.47427,4.66883,5.36691,24.21862
181,182,Palmer,CHE,MID,10.9,4.80068,4.11907,5.64511,3.95679,5.41424,4.41886,4.16081,23.93589
70,71,Kluivert,BOU,MID,5.3,3.17198,4.12228,4.39103,4.70272,4.48612,4.75275,4.44103,20.87413
90,91,Flekken,BRE,GK,4.5,4.60764,3.83872,4.57254,3.41256,4.38842,3.52040,4.81181,20.81988
247,248,Leno,FUL,GK,5.0,3.79962,3.75632,5.16383,3.39394,4.44575,3.75893,3.28876,20.55946
184,185,Sánchez,CHE,GK,4.7,4.86003,4.70652,3.13940,4.22377,3.28757,3.72189,4.23728,20.21729
382,383,Onana,MUN,GK,5.2,4.70914,3.19871,4.70818,3.31598,3.87590,3.62487,5.09267,19.80791
502,503,Son,TOT,MID,9.9,3.54874,4.26477,4.17073,3.87757,3.59901,4.85341,4.46099,19.46082
